In [ ]:
## **Immobilienpreisvorhersage mithilfe von Data Science**
**1. Business Understanding: Projektziele für mittelfristige Investorinnen und Investoren**
    Ziel der Analyse ist es, eine datengestützte Entscheidungsgrundlage für mittelfristig orientierte Investorinnen und Investoren zu schaffen. 
    Im Mittelpunkt steht die Entwicklung eines Modells, das den Marktwert einer Immobilie auf Basis ihrer Eigenschaften realistisch vorhersagt. 
    Mithilfe dieses Modells lässt sich beurteilen, ob ein Objekt im Verhältnis zu seinen Merkmalen eher günstig oder teuer ist. 
    Ein weiteres Ziel besteht darin, die wichtigsten Einflussfaktoren auf den Marktwert zu ermitteln. Die Analyse soll aufzeigen, 
    welche strukturellen und qualitativen Merkmale den Preis dominieren und in welchen Bereichen Immobilien deutliche Wertunterschiede aufweisen. 
    Dies ist entscheidend, da Investor:innen so erkennen können, welche Objektgruppen besonders hohe oder niedrige Preisniveaus aufweisen, 
    wo sich potenzielle Wertchancen ergeben und welche Merkmale den größten Einfluss auf die Preisentwicklung haben. Dieses Verständnis 
    unterstützt sie dabei, zukünftige Objekte schneller und sicherer einzuschätzen. Zusätzlich sollen Muster, Trends und Zusammenhänge 
    sichtbar werden, die für bestimmte Marktsegmente relevant sind, beispielsweise Unterschiede zwischen verschiedenen Haustypen, 
    Wohnflächen oder Zustandsklassen. Diese Erkenntnisse können Hinweise auf stabile oder besonders risikobehaftete Bereiche des Marktes liefern. 
    Insgesamt soll die Analyse Preisprognosen ermöglichen, die zentralen Werttreiber herausarbeiten und somit fundierte, datenbasierte 
    Investitionsentscheidungen unterstützen.

**2. Datenexploration und -analyse**
    